In [3]:
### IMPORT PACKAGES ###

import pandas as pd
import numpy as np

import datetime
import time
from sklearn.neighbors import KNeighborsClassifier

In [4]:
### IMPORT THE DATA ###
df = pd.read_csv('data/BPI_Challenge_2012-training.csv')

# select columns of interest
instance_column = "case concept:name"
event_column = "event concept:name"
timestamp_column = "event time:timestamp"
additionalInfo = [] # like 'event lifecycle:transition'

# putting the columns together and eliminating other columns
columns = [instance_column, event_column, timestamp_column]
columns.extend(additionalInfo)
dataset = df[columns]

# adding an index column
row_nr_column = 'row_nr'
dataset[row_nr_column] = dataset.index

# variables
timestampFormat = "%d-%m-%Y %H:%M:%S.%f"

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# checking the dataset

dataset.head()

,case concept:name,event concept:name,event time:timestamp,row_nr
0,173688,A_SUBMITTED,01-10-2011 00:38:44.546,0
1,173688,A_PARTLYSUBMITTED,01-10-2011 00:38:44.880,1
2,173688,A_PREACCEPTED,01-10-2011 00:39:37.906,2
3,173688,W_Completeren aanvraag,01-10-2011 00:39:38.875,3
4,173691,A_SUBMITTED,01-10-2011 08:08:58.256,4


In [6]:
# sorting by date

dataset.loc[:,timestamp_column] = pd.to_datetime(dataset.loc[:,timestamp_column], format=timestampFormat)
dataset = dataset.sort_values(by=timestamp_column)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [4]:
### STORAGE ###
# we want to keep track of things that happen within one instance (also called trace or case)

# instanceStorage stores: key=instance, values= [event_column, timestamp_column],
# to look up the previous event in this instance
instanceStorage = {}

# we also want to keep track of the predictionError
# errorStorage stores: key=unique integer, values= [previousEvent, previousTimestamp, predictionError, predictionErrorRatio]
errorStorage = {}


def updateStorage(case, event, timestamp, row_nr, predicted_event):
    """
    :param case: string; the case_id, so which instance this event belongs to
    :param event: string; the id of the event we're evaluating
    :param timestamp: pandas datetime; the time of the event that we're evaluating
    :param row_nr: int; unique id in the dataset
    :param predictedDuration: float; the predicted time for this event
    (which you need to store to calculate the error when you know the real time passed when the next event comes in)
    :return: previousEvent, timePassed; string, pandas datetime delta;
    the id of the previous event and the time that passed since that event, you need this information to update the model
    """
    if case in instanceStorage:

        # find the previous event
        previous_event = instanceStorage[case][0]

        # time related variables
 #       thisTimestamp = pd.to_datetime(timestamp, format=timestampFormat)
 #       previousTimestamp = pd.to_datetime(instanceStorage[case][1], format=timestampFormat)
 #       timePassed = thisTimestamp - previousTimestamp

        # add binary column based on time factors, one absolute, one based on current event
#        if timePassed >= certain_time:
#            something = True
 #       if timePassed >= scalar * regular_time_for_event
 #           something_else = True

        # since we have also saved the predicted time between the previous event and this event,
        # we can now also compute the difference
 #       if predicted_event != None:

 #           errorStorage[row_nr] = [previousEvent, previousTimestamp, predictionError, predictionErrorRatio]

    # otherwise we don't have a previous event
    else:
        previous_event = None

    # key = instance_column (so which case we're looking at), and the values are the activity and the timestamp
    instanceStorage[case] = [event, timestamp]

    return previous_event

In [4]:
### THE MODEL ###
# We start with only one feature: the historical average time between two events based on the previous event
# You don't need a second model for the test set, since you train your model on the training-set and then use it on the test-set

predictionModel = {}

def getPrediction(event):
    """
    :param event: string; the name of the activity for which you want a prediction of time until the next event
    :return: float (or None); predicted time until the next event in this instance, if there is already data on this activity,
    otherwise None
    """

    # if this activity is already in the predictionModel, then return the predicted duration
    # try & except works slightly faster then if key in dict
    # but only for valid keys, since you skip the if statement
    try:
        return predictionModel[event]

    except:
        return None

In [ ]:
def UpdateDataset(X_old,row):

    i = 0
    new_data = {}

    if i < 1000:
        i += 1

        event = row[0]

        

        new_data[event] = row[instance_column, event_column, timestamp_column, previous_event]


    else:

        X_old.append(new_data)

        i = 0
        new_data = [[]]


    return X_new

In [4]:
def updatePredictionModel(X,): #this_event_id, timePassed, weight_newtimePassed
    """
    :param this_event_id: string; the slicing-id of the current event
    :param timePassed: datatime delta; the time between the previous event in this instance and the current event
    :param weight_newtimePassed: float; how much weight should the new timePassed have in the update
    :return: None
    It will update the global variable predictionModel
    """

    'split X and y'

    i = 0

    if i < 1000:
        i += 1
    else:
        neigh_1 = KNeighborsClassifier(n_neighbors=50)
        neigh_1.fit(X, y)






    if timePassed != None:

        # if this activity is already in the predictionModel, update the model
        # try & except works slightly faster then if key in dict
        # but only for valid keys, since you skip the if statement
        try:
            predictionModel[this_event_id] = timePassed*weight_newtimePassed + predictionModel[this_event_id]*(1-weight_newtimePassed)

        except:
            predictionModel[this_event_id] = timePassed

In [4]:
def processEvent(case, event, additional, timestamp, row_nr, includeAdditional=True):